# Etapa dos Projetos no pipeline do Salic

**ISSUE:** [#32](https://github.com/lappis-unb/salic-ml/issues/32)

Este notebook é uma continuação direta deste [notebook](https://github.com/lappis-unb/salic-ml/blob/master/notebooks/exploratory/project_stage_on_pipeline-v0.1.ipynb).

### Introdução

**Etapas de Projeto:** No Salic um projeto pode passarpelas seguintes etapas, em ordem:

- Admissibilidade
- Capitação e Execução
- Prestação de Contas
  - Análise do Objeto
  - Análise Financeira

Se por algum motivo o projeto for finalizado prematuramente ele não irá passar por todas as etapas.

### Objetivos

Identificar todas as etapas que um projeto passou, incluindo as datas de início e fim em cada etapa, e as situações dentro de cada etapa.

## Prepararando os dados

**Recarregar automaticamente os módulos**

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Importando módulos python**

In [27]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from salicml.utils.dates import Dates
from salicml.models.projects import Projects

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

**Carregando os dados da tabela de Projetos**

In [28]:
cols = ['IdPRONAC', 'DtProtocolo', 'DtInicioExecucao', 'DtFimExecucao', 'Situacao']
projects_csv = os.path.join(DATA_FOLDER, 'projetos.csv')

dt_projects = pd.read_csv(projects_csv, usecols=cols, low_memory=False)
date_column = 'DtProtocolo'
dt_projects[date_column] = pd.to_datetime(dt_projects[date_column], format = Dates.DATE_INPUT_FORMAT)
dt_projects.head()

,IdPRONAC,Situacao,DtProtocolo,DtInicioExecucao,DtFimExecucao
0,1,E63,2008-09-12,2010-01-01 00:00:00,2010-12-31 00:00:00
1,2,G43,2008-09-12,NaN,NaN
2,3,K00,2008-09-12,NaN,NaN
3,4,E25,2008-09-03,NaN,NaN
4,5,E36,2008-09-12,NaN,NaN


### Filtrando os dados

É conhecido que dados muito antigos (de 1992, por exemplo) podem ser inconsistentes, então vamos filtrar os dados a partir de uma data de início (2010).

In [32]:
from datetime import datetime

START_DATE = datetime(day=1, month=1, year=2010)

dt_projects = dt_projects[dt_projects.loc[:, date_column] >= START_DATE]
print(min(dt_projects[date_column]))
dt_projects.head()

2010-01-04 00:00:00


,IdPRONAC,Situacao,DtProtocolo,DtInicioExecucao,DtFimExecucao
64477,64478,G43,2011-02-07 00:00:00,NaN,NaN
119978,120187,E36,2010-01-04 13:35:16,2010-06-01 00:00:00,2010-06-30 00:00:00
119979,120188,E68,2010-01-04 14:17:21,2010-03-01 00:00:00,2010-08-30 00:00:00
119980,120189,E63,2010-01-04 14:21:20,2010-03-01 00:00:00,2010-09-30 00:00:00
119981,120190,A17,2010-01-04 14:25:40,2010-03-01 00:00:00,2010-05-03 00:00:00


### Situção de um projeto

A tabela `projetos.csv` guarda apenas última ou a situação atual de um projeto, ou seja, não é possível a partir desta tabela construir o histórico de situações de um projeto.
O objetivo agora é investigar se as tabelas`tbProjetoSituacaoAnterior` e `HistoricoSituacao` fornecem dados suficientes para construir o histŕico de situações de um projeto.